In [1]:
from pyspark.sql import SparkSession

# Configuração do Azure Blob Storage
storage_account_name = "dadoswalney1"
storage_account_access_key = "OYkvlca1NjvKE/W3fIRn9aSqtesteteste.teste.teste.teste"
container_name = "neydados1"
# Configuração do Spark
spark = SparkSession.builder \
    .appName("PySpark Azure Blob Storage Example") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.2.1") \
    .getOrCreate()




/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [8]:
spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)
blob_file_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/walneydados.csv"


In [15]:
df = spark.read.csv(blob_file_path, header=True)

In [16]:
df.show()

+--------+-----------+-----------+--------+
|Amostra | Feature 1 | Feature 2 | Rótulo |
+--------+-----------+-----------+--------+
|       1|        2.5|        3.0|       A|
|       2|        1.0|        1.5|       B|
|       3|        3.5|        2.0|       A|
|       4|        4.0|        5.0|       B|
+--------+-----------+-----------+--------+



In [26]:
import numpy as np
numpy_db=df.toPandas().values

treino_X = np.array(numpy_db[:,0:3])
rotulos_y = np.array(numpy_db[:,3]) 

In [28]:
treino_X

array([['1', '2.5', '3.0'],
       ['2', '1.0', '1.5'],
       ['3', '3.5', '2.0'],
       ['4', '4.0', '5.0']], dtype=object)

In [29]:
rotulos_y

array(['A', 'B', 'A', 'B'], dtype=object)

In [17]:
import numpy as np

# Função para calcular a distância euclidiana entre dois pontos
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

# Classe do modelo KNN
class KNN:
    def __init__(self, k=3):
        self.k = k

    # Método de treinamento: apenas armazena os dados de treinamento
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    # Método de previsão
    def predict(self, X):
        predictions = []

        # Itera sobre cada instância de teste
        for x_test in X:
            # Calcula as distâncias entre o ponto de teste e todos os pontos de treinamento
            distances = [euclidean_distance(x_test, x_train) for x_train in self.X_train]
            
            # Ordena os índices das distâncias em ordem crescente
            k_indices = np.argsort(distances)[:self.k]
            
            # Obtém os rótulos das k instâncias mais próximas
            k_nearest_labels = [self.y_train[i] for i in k_indices]

            # Realiza a votação majoritária para determinar a classe do ponto de teste
            most_common = max(set(k_nearest_labels), key=k_nearest_labels.count)
            predictions.append(most_common)

        return predictions

In [36]:
treino_X = np.array(numpy_db[:,0:3],dtype=np.float64)
rotulos_y = np.array(numpy_db[:,3]) 

In [37]:
treino_X

array([[1. , 2.5, 3. ],
       [2. , 1. , 1.5],
       [3. , 3.5, 2. ],
       [4. , 4. , 5. ]])

In [38]:
# Exemplo de uso
if __name__ == "__main__":
    # Dados de treinamento e teste
    X_train = treino_X
    y_train = rotulos_y

    X_test = np.array([[1, 3,4], [8, 9,4], [0, 3,4], [5, 4,5], [6, 4,5]])

    # Criando e treinando o modelo KNN
    clf = KNN(k=3)
    clf.fit(X_train, y_train)

    # Realizando previsões
    predictions = clf.predict(X_test)


Predictions: ['A', 'A', 'A', 'A', 'A']


In [5]:
pip install hadoop-azure

ERROR: Could not find a version that satisfies the requirement hadoop-azure (from versions: none)
ERROR: No matching distribution found for hadoop-azure
Note: you may need to restart the kernel to use updated packages.
